# Комбинирование нескольких MCP-серверов

FastMCP позволяет объединять несколько независимых серверов в один через метод `import_server()`.

Это позволяет строить модульные системы, где каждый сервер отвечает за свою область, но все они доступны через единую точку входа.

## Шаг 1: Импорты и настройка

In [5]:
import asyncio
import os
from pathlib import Path

from dotenv import load_dotenv
from fastmcp import Client
from langchain.agents import create_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_openai import ChatOpenAI

load_dotenv()

BASE_MODEL = os.getenv("BASE_MODEL") or "qwen/qwen3-235b-a22b-2507"

## Шаг 2: Структура комбинированного сервера

У нас есть три файла:

- `math_server.py` - инструменты для математики (calculate, factorial)
- `text_server.py` - инструменты для текста (count_words, reverse_text, to_uppercase)
- `server_combined.py` - главный сервер, который импортирует оба через `mcp.import_server()`

Каждый сервер независим и может использоваться отдельно, но в комбинированном они работают вместе.

## Шаг 3: Как работает import_server()

В `server_combined.py` есть функция setup():

```python
async def setup():
    await mcp.import_server(math_server)
    await mcp.import_server(text_server)
```

Эта функция берет все инструменты из импортированных серверов и переносит их в главный сервер.

## Шаг 4: Подключение и использование

Для агента все инструменты выглядят как единый набор возможностей.

In [6]:
async def main():
    server_script = Path("server_combined.py")

    async with Client(str(server_script)) as client:
        print("Available tools from combined server:")
        tools_list = await client.list_tools()
        for tool in tools_list:
            print(f"  - {tool.name}")

        tools = await load_mcp_tools(client.session)

        llm = ChatOpenAI(model=BASE_MODEL, temperature=0)

        agent = create_agent(llm, tools)

        test_pdf = Path("../../../data/test.pdf")

        response = await agent.ainvoke(
            {
                "messages": [
                    (
                        "user",
                        f"""Perform a multi-step analysis:
1. Get statistics about the document at {test_pdf}
2. Calculate the factorial of the word count
3. Reverse the document name

Please execute all these tasks and present the results clearly.""",
                    )
                ]
            }
        )

        print("\nAgent response:")
        for message in response["messages"]:
            if (
                message.type == "ai"
                and hasattr(message, "content")
                and message.content
            ):
                print(f"\n{message.content}")

## Шаг 5: Запуск

In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
asyncio.run(main())

Available tools from combined server:
  - convert_document
  - analyze_document
  - calculate
  - count_words
  - reverse_text
  - to_uppercase

Agent response:

### Multi-Step Analysis Results:

1. **Document Statistics**  
   - Document: `test.pdf`  
   - Size: 23,354 characters  
   - Word Count: 3,655 words  
   - Lines: 547  

2. **Factorial of Word Count**  
   - Attempted to calculate: `3655!`  
   - **Result**: Could not compute due to extremely large size. Factorials of large numbers like 3655 exceed computational limits in standard environments.

3. **Reversed Document Name**  
   - Original: `test.pdf`  
   - Reversed: `fdp.tset`

All tasks completed where feasible.
